# Zadanie nr 5 - metryki w przestrzeni napisów

In [1]:
from collections import Counter
import sklearn.cluster

## 1. Metryki

<i>1. Zaimplementuj przynajmniej 3 metryki spośród wymienionych: cosinusowa, LCS, DICE, euklidesowa.</i>

* lcs

In [2]:
def lcs(x, y):
    common = [[0 for _ in range(len(y)+1)] for _ in range(len(x)+1)]
    max_lcs = 0

    for i in range(1, len(x) + 1):
        for j in range(1, len(y)+1):
            common[i][j] = common[i-1][j-1] + 1 if x[i-1] == y[j-1] else 0
            max_lcs = max(max_lcs, common[i][j])

    return 1 - max_lcs/max(len(x), len(y))

#### ngramy

In [3]:
def ngram(x, n):
    ngrams = [x[i:i+n] for i in range(len(x)-n+1)]
    return Counter(ngrams)

* dice

In [4]:
def dice(x, y, n=2):
    ngrams_x, ngrams_y = set(ngram(x, n).keys()), set(ngram(y, n).keys())
    return 1 - 2*len(ngrams_x & ngrams_y)/(len(ngrams_x)+len(ngrams_y))

* euklidesowa

In [5]:
def euclides(x, y):
    ngrams_x, ngrams_y = ngram(x, 2), ngram(y, 2)

    keys = set(ngrams_x.keys()) | set(ngrams_y.keys())
    dist = 0

    for key in keys:
        value_x = ngrams_x.get(key, 0)
        value_y = ngrams_y.get(key, 0)

        dist += (value_x - value_y)**2

    return dist**0.5

* levensheit

In [41]:
def levensheit(text_a, text_b):
    edit = [[None for _ in range(len(text_b) + 1)]
            for _ in range(len(text_a) + 1)]

    def delta(char_a, char_b): return 0 if char_a == char_b else 1

    for i in range(len(text_a) + 1):
        edit[i][0] = i

    for j in range(1, len(text_b) + 1):
        edit[0][j] = j

    for i in range(1, len(text_a)+1):
        for j in range(1, len(text_b)+1):
            edit[i][j] = min(edit[i-1][j] + 1, edit[i][j-1] + 1,
                       edit[i-1][j-1] + delta(text_a[i-1], text_b[j-1]))

    return edit[len(text_a)][len(text_b)]

In [42]:
x = 'BCDF'
y = 'ABCDEF'
print(lcs(x, y))
print(dice(x, y))
print(euclides(x, y))

0.5
0.5
2.0


## 2. Sposoby oceny jakości klasteryzacji

<i>2. Zaimplementuj przynajmniej 2 sposoby oceny jakości klasteryzacji (np. indeks Daviesa-Bouldina).</i>

In [45]:
def davies_bouldin(clusters):
    pass

In [46]:
def dunn(clusters):
    pass

## 3. Stoplista

<i>3. Stwórz stoplistę najczęściej występujących słów. </i>

In [7]:
def get_most_common_words(text, num):
    words_stats = list(Counter(text.split(' ')).items())
    words_stats.sort(reverse=True)
    return [word for word in words_stats[:num]]

## 4. Klasteryzacja

<i>4. Wykonaj klasteryzację zawartości załączonego pliku (lines.txt) przy użyciu przynajmniej 2 algorytmów oraz metryk zaimplementowanych w pkt. 1. i metryki Levenshteina. Każda linia to adres pocztowy firmy, różne sposoby zapisu tego samego adresu powinny się znaleźć w jednym klastrze. </i>

In [29]:
def cluster(texts, metric):
    distances = [[metric(texts[i], texts[j]) for i in range(len(texts))] for j in range(len(texts))]
    clustering = sklearn.cluster.DBSCAN(eps=20, min_samples=1).fit(distances)
    return clustering.labels_

## 5. Testy

<i> 5. Porównaj jakość wyników sposobami zaimplementowanymi w pkt. 2. </i>

In [30]:
def get_example_lines(num):
    with open('lines.txt', 'r') as file:
        lines = list(file)
    lines = lines[:num]

    with open('clusters.txt', 'r') as file:
        clusters = list(file)
    
    correct_clustering = []
    cluster_num = 0
    for line in lines:
        cluster = 0
        for cluster_line in clusters:
            if cluster_line[0] == '#':
                cluster += 1
            elif cluster_line == line:
                correct_clustering.append(cluster)
    
    return lines, correct_clustering

In [31]:
lines, correct_labels = get_example_lines(500)

In [32]:
labels = cluster(lines, euclides)

In [33]:
print(labels)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  12  12  12  12  12
  12  13  13  14  15  16  17   8  18  19  19  20  21  22  23  23  24  25
   8  26   8  27  17   8   8  28   8  29  30  31  32  32   8  33  34  34
  35  36  37  36  38  39  38  38  39  39  40  41  40  42  42  42  43  44
  45  45  46  47  48  49  50  51   8  52   8  53  53  54  55  56   2  57
  58   8   8   8   8   8  59  60  60  60  60  60  61   4   4   4   4   4
  62   4   4  63   4   4  64  65  66  67  67  68  58   8  69  69  70  71
  72  73   8  74  75  45  76   8  77  78   8   8  79  79  80  81  82  82
  83  83  84  84  83   8  85   7   7  86   8  87  82  82  88  88  89  90
  91  92  92  92  93  45   8   8   8   8   8  94  95  96  96  97  97  98
  96   8  99 100 100 100 101 101 101 101 102 103 104 105 106   8 107 108
 109 110 111 112   8 113 113   8 114 115 115 116 116 116 117 118 119 120
 121   8 122 122 122  90  93 123 123 123 123   8   8 124 125  52 126 127
 127 128 126 126 127 129 130 130 129 129 129 129 12

In [35]:
def renumber_labels(labels):
    seen = {}
    current = 0
    new_labels = []
    for label in labels:
        if label in seen:
            new_labels.append(seen[label])
        else:
            new_labels.append(current)
            seen[label] = current
            current += 1
    
    return new_labels      

In [36]:
print(renumber_labels(correct_labels))

[0, 1, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 11, 11, 11, 11, 11, 11, 12, 12, 13, 14, 14, 15, 16, 17, 18, 18, 19, 20, 21, 22, 22, 23, 24, 25, 26, 27, 28, 29, 29, 30, 31, 32, 33, 34, 35, 36, 36, 37, 38, 38, 38, 39, 39, 39, 39, 40, 40, 40, 40, 40, 40, 41, 41, 41, 42, 42, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 56, 57, 58, 59, 2, 60, 61, 62, 63, 64, 63, 64, 65, 65, 65, 65, 65, 66, 67, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 68, 69, 70, 70, 70, 71, 72, 73, 74, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 89, 90, 91, 27, 27, 26, 26, 26, 26, 26, 92, 93, 6, 6, 94, 53, 95, 27, 27, 26, 26, 96, 91, 91, 91, 91, 91, 91, 97, 98, 99, 99, 100, 99, 101, 102, 102, 102, 102, 102, 103, 102, 53, 104, 105, 105, 105, 106, 106, 106, 106, 106, 106, 107, 107, 93, 108, 109, 26, 110, 111, 112, 113, 114, 115, 115, 53, 93, 116, 116, 117, 117, 117, 118, 119, 120, 121, 122, 123, 124, 124, 124, 125, 125, 126, 126, 126, 126, 99, 127, 128, 129, 130, 131, 131, 131, 131, 131, 131, 131, 132, 13

<i>6. Czy masz jakiś pomysł na poprawę jakości klasteryzacji w tym zadaniu? </i>

Sprawozdanie powinno zawierać porównanie wyników wszystkich metryk z użyciem stoplisty i bez.
Można jako wzorcową klasteryzację użyć pliku clusters.txt.

## Wnioski

- 
- 

M. Hawryluk 15.05.2021